In [2]:
from datetime import datetime
import itertools
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.callbacks import ModelCheckpoint
from keras_preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

from sklearn.utils import shuffle
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Embedding, Conv1D, GlobalMaxPooling1D, Flatten, LSTM, \
    Bidirectional, MaxPooling1D
from keras.preprocessing import text, sequence
from keras import utils
import pandas as pd

from utility.train_data_loader import load_train_data


testData = pd.read_csv("../data/test.csv")
dictData = pd.read_csv("../data/kata_dasar_kbbi.csv")
categories_file = open("../data/categories.json", "r")
categories = json.load(categories_file)
inverted_categories_mobile = {v: k.lower() for k, v in categories['Mobile'].items()}
inverted_categories_fashion = {v: k.lower() for k, v in categories['Fashion'].items()}
inverted_categories_beauty = {v: k.lower() for k, v in categories['Beauty'].items()}

all_subcategories = {k.lower(): v for k, v in categories['Mobile'].items()}
all_subcategories.update({k.lower(): v for k, v in categories['Fashion'].items()})
all_subcategories.update({k.lower(): v for k, v in categories['Beauty'].items()})

# Main settings
plot_history_check = True
gen_test = True
max_length = 35  # 32 is max word in train
max_words = 2500
num_classes = len(all_subcategories)
# Training for more epochs will likelval-acc after 10 epochs: 0.71306y lead to overfitting on this dataset
# You can try tweaking these hyperparamaters when using this model with your own data
batch_size = 256
epochs = 10

print(all_subcategories)
print("no of categories: " + str(num_classes))

category_mapping = {
    'fashion_image': 'Fashion',
    'beauty_image': 'Beauty',
    'mobile_image': 'Mobile',
}
directory_mapping = {
    'Fashion': 'fashion_image',
    'Beauty': 'beauty_image',
    'Mobile': 'mobile_image',
}

trainData = load_train_data()

Using TensorFlow backend.


{'others mobile & tablet': 35, 'smartfren': 53, 'infinix': 40, 'brandcode': 39, 'icherry': 52, 'advan': 45, 'iphone': 31, 'realme': 51, 'motorola': 49, 'maxtron': 56, 'nokia': 38, 'xiaomi': 34, 'mito': 46, 'sony': 33, 'spc': 57, 'lenovo': 37, 'alcatel': 55, 'samsung': 32, 'vivo': 42, 'evercoss': 44, 'strawberry': 50, 'blackberry': 36, 'asus': 43, 'honor': 54, 'oppo': 41, 'huawei': 47, 'sharp': 48, 'wedding dress': 23, 'shirt': 27, 'casual dress': 18, 'maxi dress': 20, 'big size dress': 24, 'bodycon dress': 22, 'party dress': 19, 'blouse': 26, 'tshirt': 25, 'crop top': 29, 'tanktop': 28, 'others': 17, 'a line dress': 21, 'big size top': 30, 'foundation': 1, 'face palette': 0, 'concealer': 7, 'lip gloss': 14, 'blush on': 2, 'highlighter': 8, 'bb & cc cream': 5, 'other face cosmetics': 4, 'lip tint': 13, 'bronzer': 11, 'lip liner': 15, 'powder': 3, 'setting spray': 10, 'primer': 9, 'contour': 6, 'other lip cosmetics': 16, 'lipstick': 12}
no of categories: 58
custom train data used


In [12]:
trainData.head()

,itemid,title,Category,image_path,item_category
487927,723561441,atas wanita kemeja lengan panjang ij lg stripe...,27,fashion_image/7d53cdc75ba4ad2bccb682bf33701627...,shirt
410094,1718626886,best seller black lace sleeve dress pesta mini...,18,fashion_image/8e41b179b48a73d22db77934c0db800b...,casual dress
86696,838724637,catrice hd liquid coverage foundation,1,beauty_image/bbae6b05a25f3753d3b45e81ebf80e34.jpg,foundation
293146,1660979197,dress bodycon mini sexy model off shoulder len...,21,fashion_image/9a4e2c84a93941c6bf21cf8ec9551bfe...,a line dress
158120,504154458,cream glansie paket normal dr fajar,4,beauty_image/6f1942777ee11792996a4c410df73f4c.jpg,other face cosmetics


In [3]:
# Shuffle train data
trainData = shuffle(trainData)

max_data_size = int(len(trainData) * 1)
train_data_size = int(max_data_size * .9)
train_data_step = 1
validate_data_step = 1
print(train_data_size, max_data_size)

train_texts = trainData['title'][:train_data_size:train_data_step]
valid_texts = trainData['title'][train_data_size::train_data_step]
train_tags = trainData['Category'][:train_data_size:train_data_step]
valid_tags = trainData['Category'][train_data_size::train_data_step]
test_texts = testData['title']
print(len(train_texts), len(train_tags))

y = train_tags.values

tokenize = text.Tokenizer(num_words=max_words, char_level=False)
tokenize.fit_on_texts(train_texts)  # only fit on train
x_train = tokenize.texts_to_sequences(train_texts)
x_valid = tokenize.texts_to_sequences(valid_texts)
x_test = tokenize.texts_to_sequences(test_texts)

word_index = tokenize.word_index

# Pad sequences with zeros
x_train = pad_sequences(x_train, padding='post', maxlen=max_length)
x_valid = pad_sequences(x_valid,padding='post',maxlen=max_length)
x_test = pad_sequences(x_test, padding='post', maxlen=max_length)

y_train = train_tags.values
y_valid = valid_tags.values
vocab_size = len(tokenize.word_index) + 1
print(vocab_size)


599953 666615
599953 599953
41558


In [11]:
x_train[10000]


array([ 32,  51,  15, 213,  13, 322, 601,   4,  81,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0])

In [2]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers


def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)

    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)

    if is_neural_net:
        predictions = predictions.argmax(axis=-1)

    return metrics.accuracy_score(predictions, valid_y)

In [3]:
# split the dataset into training and validation datasets
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainData['title'], trainData['Category'])

In [4]:
# label encode the target variable
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [5]:
# create a count vectorizer object
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(trainData['title'])
print("")
# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [6]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(trainData['title'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

In [7]:
# ngram level tf-idf
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(trainData['title'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

In [8]:
# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(trainData['title'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x)
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x)

In [11]:
xgbclassifier = xgboost.XGBClassifier(verbosity = 3)

In [9]:
# Extereme Gradient Boosting on Count Vectors
accuracy = train_model(xgbclassifier, xtrain_count.tocsc(), train_y, xvalid_count.tocsc())
print ("Xgb, Count Vectors: ", accuracy)

Xgb, Count Vectors:  0.6870942191606562


In [12]:
# Extereme Gradient Boosting on Word Level TF IDF Vectors
accuracy = train_model(xgbclassifier, xtrain_tfidf.tocsc(), train_y, xvalid_tfidf.tocsc())
print ("Xgb, WordLevel TF-IDF: ", accuracy)

Xgb, WordLevel TF-IDF:  0.6889003564270885


In [ ]:
# Extereme Gradient Boosting on Character Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(verbosity = 3), xtrain_tfidf_ngram_chars.tocsc(), train_y, xvalid_tfidf_ngram_chars.tocsc())
print ("Xgb, CharLevel Vectors: ", accuracy)